In [31]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.api import VAR, DynamicVAR
from statsmodels.tsa.base.datetools import dates_from_str

In [47]:
# Check whether one time series G-causes the other
#    model:
#           y_t = Sum_{j=1}^p A_{11, j} x_{t-j} + Sum_{j=1}^p A_{12, j} y_{t-j} + Error
#    inputs:
#           x_t: regressor time trend (pandas series of length T)
#           y_t: dependent variable time trend (pandas series of length T)
#    output:
#           logarithm of F-statistic from F-Test with null hypothesis A_{12}| = 0
def granger_causes(x_t, y_t):
    data = pd.concat([x_t, y_t], axis=1)
    data.index = pd.DatetimeIndex(range(0,len(x_t))) # currently uses index as ns
    model = VAR(data)
    results = model.fit(2)
    results.test_causality(list(data)[1], [list(data)[0])], kind='f')
    

In [48]:
granger_causes(x_t, y_t)

Granger causality f-test
   Test statistic   Critical Value          p-value        df
-------------------------------------------------------------
        22.921848         3.018743            0.000  (2, 392)
H_0: ['realcons'] do not Granger-cause realgdp
Conclusion: reject H_0 at 5.00% significance level


In [6]:
mdata = sm.datasets.macrodata.load_pandas().data

In [37]:
x_t = mdata['realgdp']
y_t = mdata['realcons']
z_t = mdata['realinv']

In [52]:
data = pd.concat([x_t, y_t, z_t], axis=1)

#data.index = pd.DatetimeIndex(range(0,len(x_t))) # currently uses index as ns

# test
dates = mdata[['year', 'quarter']].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
quarterly = dates_from_str(quarterly)
data.index = pd.DatetimeIndex(quarterly)

cdata = data.diff().dropna()
model = VAR(cdata)
results = model.fit(2)
results.summary()

print(list(data)[1])

realcons


       realgdp  realcons   realinv
0     2710.349    1707.4   286.898
1     2778.801    1733.7   310.859
2     2775.488    1751.8   289.226
3     2785.204    1753.7   299.356
4     2847.699    1770.5   331.722
5     2834.390    1792.9   298.152
6     2839.022    1785.8   296.375
7     2802.616    1788.2   259.764
8     2819.264    1787.7   266.405
9     2872.005    1814.3   286.246
10    2918.419    1823.1   310.227
11    2977.830    1859.6   315.463
12    3031.241    1879.4   334.271
13    3064.709    1902.5   331.039
14    3093.047    1917.9   336.962
15    3100.563    1945.1   325.650
16    3141.087    1958.2   343.721
17    3180.447    1976.9   348.730
18    3240.332    2003.8   360.102
19    3264.967    2020.6   364.534
20    3338.246    2060.5   379.523
21    3376.587    2096.7   377.778
22    3422.469    2135.2   386.754
23    3431.957    2141.2   389.910
24    3516.251    2188.8   429.145
25    3563.960    2213.0   429.119
26    3636.285    2251.0   444.444
27    3724.014    23

In [25]:
dates = mdata[['year', 'quarter']].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
from statsmodels.tsa.base.datetools import dates_from_str
quarterly = dates_from_str(quarterly)
print(quarterly)

[datetime.datetime(1959, 3, 31, 0, 0), datetime.datetime(1959, 6, 30, 0, 0), datetime.datetime(1959, 9, 30, 0, 0), datetime.datetime(1959, 12, 31, 0, 0), datetime.datetime(1960, 3, 31, 0, 0), datetime.datetime(1960, 6, 30, 0, 0), datetime.datetime(1960, 9, 30, 0, 0), datetime.datetime(1960, 12, 31, 0, 0), datetime.datetime(1961, 3, 31, 0, 0), datetime.datetime(1961, 6, 30, 0, 0), datetime.datetime(1961, 9, 30, 0, 0), datetime.datetime(1961, 12, 31, 0, 0), datetime.datetime(1962, 3, 31, 0, 0), datetime.datetime(1962, 6, 30, 0, 0), datetime.datetime(1962, 9, 30, 0, 0), datetime.datetime(1962, 12, 31, 0, 0), datetime.datetime(1963, 3, 31, 0, 0), datetime.datetime(1963, 6, 30, 0, 0), datetime.datetime(1963, 9, 30, 0, 0), datetime.datetime(1963, 12, 31, 0, 0), datetime.datetime(1964, 3, 31, 0, 0), datetime.datetime(1964, 6, 30, 0, 0), datetime.datetime(1964, 9, 30, 0, 0), datetime.datetime(1964, 12, 31, 0, 0), datetime.datetime(1965, 3, 31, 0, 0), datetime.datetime(1965, 6, 30, 0, 0), datet